In [1]:
import time
from datetime import date
import pandas as pd
import os
import glob
import pycytominer
import sys
import CBE_utils as CBE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import gc

In [2]:
import importlib
importlib.reload(CBE)

<module 'CBE_utils' from '/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/notebooks_revision/CBE_utils.py'>

In [3]:
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/output/"

annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"
feature_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/reduced_features/"

In [4]:
def list_folders(directory):
    try:
        # List all entries in the specified directory
        entries = os.listdir(directory)
        
        # Filter out non-folder entries
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        return folders
    
    except FileNotFoundError:
        
        return f"The directory '{directory}' does not exist."
    
    except PermissionError:
        
        return f"Permission denied to access the directory '{directory}'."

In [5]:
# TODO: Check how many unique plates exist
source_list = list_folders(input_path)

files = []

for source in source_list:
    
    print(source)

    source_path = os.path.join(input_path, source)

    plate_list = list_folders(source_path)

    

    for plate in plate_list:
        
        plate_path = os.path.join(source_path, plate)

        

        raw_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4].csv"
        if glob.glob(plate_path + os.sep + raw_pattern , recursive=True):

            raw_file_exists = True

        else:

            raw_file_exists = False

        norm_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4]_mad_robustize.csv"

        if glob.glob(plate_path + os.sep + norm_pattern  , recursive=True):

            norm_file_exits = True
            
        else:

            norm_file_exits = False
       
        processed_pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4]_mad_robustize_reduced-corr.csv"
        
        if glob.glob(plate_path + os.sep + processed_pattern , recursive=True):

            reduced_file_exists = True
            
        else:

            reduced_file_exists = False
       
        files.append({'imaging_site': source, 
                     'plate': plate, 
                     'raw_file': raw_file_exists, 
                     'norm_file': norm_file_exits, 
                     'reduced_file': reduced_file_exists})

IMTM
USC
FMP
MEDINA


In [6]:
file_check = pd.DataFrame(files)

In [7]:
file_check['batch'], file_check['plate_name'], file_check['replicate'] = file_check['plate'].str.split('_', n=2).str

<ipython-input-7-50efb533959c>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  file_check['batch'], file_check['plate_name'], file_check['replicate'] = file_check['plate'].str.split('_', n=2).str


In [8]:
file_check

,imaging_site,plate,raw_file,norm_file,reduced_file,batch,plate_name,replicate
0,IMTM,2024-09-05_C1269_R2,True,True,True,2024-09-05,C1269,R2
1,IMTM,2025-01-09_C1242_R2,True,True,True,2025-01-09,C1242,R2
2,IMTM,2024-11-06_C1288_R1,True,True,True,2024-11-06,C1288,R1
3,IMTM,2024-06-25_C1240_R4,True,True,True,2024-06-25,C1240,R4
4,IMTM,2025-01-13_C1242_R3,True,True,True,2025-01-13,C1242,R3
...,...,...,...,...,...,...,...,...
959,MEDINA,2024-10-28_C1041_R3,True,True,True,2024-10-28,C1041,R3
960,MEDINA,2025-02-10_C1065_R4,True,True,True,2025-02-10,C1065,R4
961,MEDINA,2025-01-23_C1066_R3,True,True,True,2025-01-23,C1066,R3
962,MEDINA,2024-09-25_C1038_R2,True,True,True,2024-09-25,C1038,R2


In [9]:
def file_checker(file_check_dataframe, site_string):

    site_files = file_check_dataframe[file_check_dataframe['imaging_site'] == site_string] 

    print( site_string + ' file check:')
    print(str(site_files['raw_file'].count()) + ' raw file(s)')
    print(str(site_files['norm_file'].count()) + ' normalized file(s)')
    print(str(site_files['reduced_file'].count()) + ' feature reduced file(s)')

In [10]:
file_checker(file_check, 'IMTM')

IMTM file check:
247 raw file(s)
247 normalized file(s)
247 feature reduced file(s)


In [11]:
file_checker(file_check, 'FMP')

FMP file check:
184 raw file(s)
184 normalized file(s)
184 feature reduced file(s)


In [12]:
file_checker(file_check, 'USC')

USC file check:
283 raw file(s)
283 normalized file(s)
283 feature reduced file(s)


In [13]:
file_checker(file_check, 'MEDINA')

MEDINA file check:
250 raw file(s)
250 normalized file(s)
250 feature reduced file(s)


In [67]:
# Foldername in input is carrying the batch date
input = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/input/"

In [69]:
# TODO: Check how many unique plates exist
source_list = list_folders(input)

date_files = []

for source in source_list:
    
    print(source)

    source_path = os.path.join(input, source)

    plate_list = list_folders(source_path)

    for plate in plate_list:

        print(plate)

        plate_path = os.path.join(source_path, plate)

        date_list = list_folders(plate_path)
        
        for date in  date_list:


            print(date)
        
            date_path = os.path.join(plate_path, date)

            raw_pattern = "*CP_Profiles_Aggregated.csv"

            if glob.glob(date_path + os.sep + raw_pattern , recursive=True):

                raw_file_exists = True

            else:

                raw_file_exists = False
       
            date_files.append({'imaging_site': source,
                          'plate': plate,
                          'batch_date' : date,
                          'raw_file': raw_file_exists})

IMTM
C1284R2
241106
C1235R4
240612
C1244R2
240710
C1270R2
240816
C1230R4
240529
C1275R3
240906
C1242R3
250110
C1278R4
241129
C1269R4
240823
C1275R1
240822
C1262R4
160824
C1282R4
241108
C1263R1
240724
C1234R1
250110
C1242R1
250114
C1238R4
240619
C1282R1
241025
C1283R3
241106
C1257R1
240717
C1245R1
240703
C1232R2
250108
C1286R2
241106
C1233R4
240605
C1262R1
240724
C1281R3
241106
C1256R1
240717
C1257R3
240725
C1232R1
250114
C1274R4
241025
C1254R1
240710
C1240R3
250110
C1279R3
241106
C1260R2
240724
C1286R4
241128
C1261R3
240725
C1268R3
240822
C1269R1
240815
C1264R1
240724
C1245R2
240710
C1242R2
250108
C1266R1
240815
C1270R3
240822
C1252R4
240719
C1265R3
240815
C1271R2
240816
C1277R3
240906
C1276R2
240904
C1230R3
250110
C1237R4
240619
C1288R3
241115
C1291R3
250117
C1274R1
240822
C1272R3
240822
C1269R2
240816
C1290R1
241129
C1280R2
241101
C1265R4
160824
C1253R3
240717
C1240R4
240621
C1252R1
240710
C1260R3
240725
C1290R2
241204
C1245R4
240712
C1289R2
241204
C1253R4
240719
C1246R4
240712
C1252

In [72]:
file_check = pd.DataFrame(date_files)
file_check.head()

,imaging_site,plate,batch_date,raw_file
0,IMTM,C1284R2,241106,True
1,IMTM,C1235R4,240612,True
2,IMTM,C1244R2,240710,True
3,IMTM,C1270R2,240816,True
4,IMTM,C1230R4,240529,True


In [74]:
file_check[['Plate', 'Replicate']] = file_check['plate'].str.split('R', expand=True)

In [77]:
file_check['Replicate'] = 'R' + file_check['Replicate'] 

In [79]:
file_check['Plate_map_name'] = file_check['Plate']  + '_' + file_check['Replicate'] 

In [80]:
file_check.head()

,imaging_site,plate,batch_date,raw_file,Plate,Replicate,Plate_map_name
0,IMTM,C1284R2,241106,True,C1284,R2,C1284_R2
1,IMTM,C1235R4,240612,True,C1235,R4,C1235_R4
2,IMTM,C1244R2,240710,True,C1244,R2,C1244_R2
3,IMTM,C1270R2,240816,True,C1270,R2,C1270_R2
4,IMTM,C1230R4,240529,True,C1230,R4,C1230_R4


In [81]:
IMTM_site_files = file_check[file_check['imaging_site'] == 'IMTM'] 

In [82]:
selected_IMTM_site_files = IMTM_site_files[["Plate_map_name", "Plate", "Replicate", "batch_date"]]

In [84]:
selected_IMTM_site_files = selected_IMTM_site_files.rename(columns={'batch_date': 'batch_date_corr'})

In [85]:
qc_path = '/home/schmiedc/Documents/IMTM_QC_Analyzed.xlsx'
qc_df = pd.read_excel(qc_path)

In [86]:
merged_df = pd.merge(qc_df, selected_IMTM_site_files, how="outer", on=["Plate_map_name", "Plate", "Replicate"])


In [87]:
merged_df.head()

,Plate_map_name,Plate,Replicate,Compounds,Library,Library subpart,Library internal,Partner_site,Batch Date,Site QC,...,visible artifacts,heatmap,cell_number,intra_plate,intra_batch,inter_batch,QC SUMMARY,Note,Rerun,batch_date_corr
0,C1230_R1,C1230,R1,352,ECBL,Diversity,Enamine,IMTM,240529.0,Mitotracker 1st 4 rows weird,...,dispenser stripes DNA,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250114
1,C1231_R1,C1231,R1,352,ECBL,Diversity,Enamine,IMTM,240531.0,Mitotracker weird,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250114
2,C1232_R1,C1232,R1,352,ECBL,Diversity,Enamine,IMTM,240605.0,ER 1st and 3rd row small intensity,...,NaN,pass,low cell number,NaN,NaN,NaN,NaN,NaN,NaN,250114
3,C1233_R1,C1233,R1,352,ECBL,Diversity,Enamine,IMTM,240605.0,Mitotracker rows,...,NaN,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250114
4,C1234_R1,C1234,R1,352,ECBL,Diversity,Enamine,IMTM,240607.0,Mitotracker rows,...,weak dispenser ER,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250110


In [88]:
path_corr = '/home/schmiedc/Documents/output2.xlsx'
# Save DataFrame to Excel
merged_df.to_excel(path_corr, index=False)